<a href="https://colab.research.google.com/github/dinarsadykow/cv_hw/blob/main/Stepik_SQL_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL stepik
https://stepik.org/course/63054?auth=login

In [1]:
%%capture

import json

# ------------------------------------------------------
# pySpark Tutorial from https://towardsdatascience.com/pyspark-on-google-colab-101-d31830b238be

!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pickle5
!pip install pyspark_dist_explore

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 --executor-memory 4g --driver-memory 3g pyspark-shell'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, LongType

spark = (
      SparkSession.builder\
        .master("local")\
        .appName("sdr")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.maxResultSize", "4g")\
        #.config("spark.sql.crossJoin.enabled", "true")\
         .config("spark.sql.catalogImplementation","hive")
         .config("spark.sql.hive.convertMetastoreParquet","false")
        .getOrCreate()
      )

from pyspark.ml.feature import Tokenizer, HashingTF, IDF

import pyspark.sql.functions as F
import pyspark.sql.types as T

import gc
import re
from datetime import datetime

import pickle5 as pickle
import json

from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

spark

# Data Load/Send to Spark: SQL Table

In [2]:
import pandas as pd

t = """('Баранов П.Е.','Москва',700 , '2020-01-12', '2020-01-17'),
('Абрамова К.А.','Владивосток',450 , '2020-01-14', '2020-01-27'),
('Семенов И.В.','Москва',700 , '2020-01-23', '2020-01-31'),
('Ильиных Г.Р.','Владивосток', 450, '2020-01-12', '2020-02-02'),
('Колесов С.П.','Москва',700 , '2020-02-01', '2020-02-06'),
('Баранов П.Е.','Москва', 700, '2020-02-14', '2020-02-22'),
('Абрамова К.А.','Москва', 700, '2020-02-23', '2020-03-01'),
('Лебедев Т.К.','Москва', 700, '2020-03-03', '2020-03-06'),
('Колесов С.П.','Новосибирск',450 , '2020-02-27', '2020-03-12'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-03-29', '2020-04-05'),
('Абрамова К.А.','Москва',700 , '2020-04-06', '2020-04-14'),
('Баранов П.Е.','Новосибирск',450 , '2020-04-18', '2020-05-04'),
('Лебедев Т.К.','Томск',450 , '2020-05-20', '2020-05-31'),
('Семенов И.В.','Санкт-Петербург',700 , '2020-06-01', '2020-06-03'),
('Абрамова К.А.','Санкт-Петербург', 700, '2020-05-28', '2020-06-04'),
('Федорова А.Ю.','Новосибирск',450 , '2020-05-25', '2020-06-04'),
('Колесов С.П.','Новосибирск', 450, '2020-06-03', '2020-06-12'),
('Федорова А.Ю.','Томск', 450, '2020-06-20', '2020-06-26'),
('Абрамова К.А.','Владивосток', 450, '2020-07-02', '2020-07-13'),
('Баранов П.Е.','Воронеж', 450, '2020-07-19', '2020-07-25')"""

df_pd = pd.DataFrame( [ x \
                .replace(')','') \
                .replace('(','') \
                .replace("'",'') \
                .split(',') for x in t.split('\n')] ).drop(5,axis=1)
df_pd.columns = ['name','city','per_diem','date_first','date_last']
df_pd['per_diem'] = df_pd['per_diem'].astype('int32')

df_pd['date_first'] = pd.to_datetime( df_pd['date_first'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')
df_pd['date_last'] = pd.to_datetime( df_pd['date_last'].map(lambda x: x.strip() )
                                    ,format='%Y-%m-%d'
                                    , errors='ignore')

# df_pd


mySchema = StructType([
                       StructField("name", T.StringType(), True)\
                       ,StructField("city", T.StringType(), True)\
                       ,StructField("per_diem", T.IntegerType(), True)\
                       ,StructField("date_first", T.TimestampType(), True)\
                       ,StructField("date_last", T.TimestampType(), True)
                       ])

df_sp = spark \
      .createDataFrame(df_pd ,schema=mySchema ) \
      .withColumn('date_first', F.to_date(F.col('date_first')) ) \
      .withColumn('date_last', F.to_date(F.col('date_last')) )

df_sp.createOrReplaceTempView("trip")

df_sp.printSchema()
df_sp.show()

root
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- per_diem: integer (nullable = true)
 |-- date_first: date (nullable = true)
 |-- date_last: date (nullable = true)

+-------------+---------------+--------+----------+----------+
|         name|           city|per_diem|date_first| date_last|
+-------------+---------------+--------+----------+----------+
| Баранов П.Е.|         Москва|     700|2020-01-12|2020-01-17|
|Абрамова К.А.|    Владивосток|     450|2020-01-14|2020-01-27|
| Семенов И.В.|         Москва|     700|2020-01-23|2020-01-31|
| Ильиных Г.Р.|    Владивосток|     450|2020-01-12|2020-02-02|
| Колесов С.П.|         Москва|     700|2020-02-01|2020-02-06|
| Баранов П.Е.|         Москва|     700|2020-02-14|2020-02-22|
|Абрамова К.А.|         Москва|     700|2020-02-23|2020-03-01|
| Лебедев Т.К.|         Москва|     700|2020-03-03|2020-03-06|
| Колесов С.П.|    Новосибирск|     450|2020-02-27|2020-03-12|
| Семенов И.В.|Санкт-Петербург|     700|2020-0

#Задание
Вывести из таблицы trip информацию о командировках тех сотрудников, фамилия которых заканчивается на букву «а», в отсортированном по убыванию даты последнего дня командировки виде. В результат включить столбцы name, city, per_diem, date_first, date_last.

In [3]:
sql_str = """ select * from trip """
spark.sql(sql_str).toPandas()

,name,city,per_diem,date_first,date_last
0,Баранов П.Е.,Москва,700,2020-01-12,2020-01-17
1,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27
2,Семенов И.В.,Москва,700,2020-01-23,2020-01-31
3,Ильиных Г.Р.,Владивосток,450,2020-01-12,2020-02-02
4,Колесов С.П.,Москва,700,2020-02-01,2020-02-06
5,Баранов П.Е.,Москва,700,2020-02-14,2020-02-22
6,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
7,Лебедев Т.К.,Москва,700,2020-03-03,2020-03-06
8,Колесов С.П.,Новосибирск,450,2020-02-27,2020-03-12
9,Семенов И.В.,Санкт-Петербург,700,2020-03-29,2020-04-05


In [4]:
sql_161 = """SELECT
    name
    , city
    , per_diem
    , date_first
    , date_last

FROM trip

WHERE name LIKE '%а %'
ORDER BY date_last DESC"""

spark.sql(sql_161).toPandas()

,name,city,per_diem,date_first,date_last
0,Абрамова К.А.,Владивосток,450,2020-07-02,2020-07-13
1,Федорова А.Ю.,Томск,450,2020-06-20,2020-06-26
2,Абрамова К.А.,Санкт-Петербург,700,2020-05-28,2020-06-04
3,Федорова А.Ю.,Новосибирск,450,2020-05-25,2020-06-04
4,Абрамова К.А.,Москва,700,2020-04-06,2020-04-14
5,Абрамова К.А.,Москва,700,2020-02-23,2020-03-01
6,Абрамова К.А.,Владивосток,450,2020-01-14,2020-01-27


#Задание
Вывести в алфавитном порядке фамилии и инициалы тех сотрудников, которые были в командировке в Москве.

In [5]:
sql_162 = """SELECT
    name

FROM trip

WHERE city = 'Москва'
ORDER BY name ASC"""

spark.sql(sql_162).toPandas()

,name
0,Абрамова К.А.
1,Абрамова К.А.
2,Баранов П.Е.
3,Баранов П.Е.
4,Колесов С.П.
5,Лебедев Т.К.
6,Семенов И.В.


#Задание
Для каждого города посчитать, сколько раз сотрудники в нем были.  Информацию вывести в отсортированном в алфавитном порядке по названию городов. Вычисляемый столбец назвать Количество. 

In [6]:
sql_163 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY city ASC"""

spark.sql(sql_163).toPandas()

,city,Kolichestvo
0,Владивосток,3
1,Воронеж,1
2,Москва,7
3,Новосибирск,4
4,Санкт-Петербург,3
5,Томск,2


#Задание
Вывести два города, в которых чаще всего были в командировках сотрудники. Вычисляемый столбец назвать Количество.

In [7]:
sql_164 = """SELECT 
    city
    , COUNT(*) AS Kolichestvo

FROM trip

GROUP BY city
ORDER BY Kolichestvo DESC
LIMIT 2"""

spark.sql(sql_164).toPandas()

,city,Kolichestvo
0,Москва,7
1,Новосибирск,4


#Задание
Вывести информацию о командировках во все города кроме Москвы и Санкт-Петербурга (фамилии и инициалы сотрудников, город ,  длительность командировки в днях, при этом первый и последний день относится к периоду командировки). Последний столбец назвать Длительность. Информацию вывести в упорядоченном по убыванию длительности поездки, а потом по убыванию названий городов (в обратном алфавитном порядке).

In [8]:
sql_165 = """SELECT
    name
    , city
    , DATEDIFF(date_last,date_first)+1 AS Dlitelnost

FROM trip
WHERE city NOT IN ('Москва','Санкт-Петербург')
ORDER BY Dlitelnost DESC, city DESC"""

spark.sql(sql_165).toPandas()

,name,city,Dlitelnost
0,Ильиных Г.Р.,Владивосток,22
1,Баранов П.Е.,Новосибирск,17
2,Колесов С.П.,Новосибирск,15
3,Абрамова К.А.,Владивосток,14
4,Лебедев Т.К.,Томск,12
5,Абрамова К.А.,Владивосток,12
6,Федорова А.Ю.,Новосибирск,11
7,Колесов С.П.,Новосибирск,10
8,Федорова А.Ю.,Томск,7
9,Баранов П.Е.,Воронеж,7


#Задание
Вывести информацию о командировках сотрудника(ов), которые были самыми короткими по времени. В результат включить столбцы name, city, date_first, date_last.

In [9]:
sql_166 = """SELECT

    name, city, date_first, date_last

FROM trip

WHERE (DATEDIFF(date_last,date_first)+1)
            =(
                SELECT MIN(DATEDIFF(date_last,date_first))+1 FROM trip
                )"""

spark.sql(sql_166).toPandas()

,name,city,date_first,date_last
0,Семенов И.В.,Санкт-Петербург,2020-06-01,2020-06-03


#Задание
Вывести информацию о командировках, начало и конец которых относятся к одному месяцу (год может быть любой). В результат включить столбцы name, city, date_first, date_last. Строки отсортировать сначала  в алфавитном порядке по названию города, а затем по фамилии сотрудника .

In [10]:
sql_167 = """

SELECT

    name, city, date_first, date_last

FROM trip

WHERE MONTH(date_last)=MONTH(date_first)

ORDER BY city, name

"""

spark.sql(sql_167).toPandas()

,name,city,date_first,date_last
0,Абрамова К.А.,Владивосток,2020-01-14,2020-01-27
1,Абрамова К.А.,Владивосток,2020-07-02,2020-07-13
2,Баранов П.Е.,Воронеж,2020-07-19,2020-07-25
3,Абрамова К.А.,Москва,2020-04-06,2020-04-14
4,Баранов П.Е.,Москва,2020-01-12,2020-01-17
5,Баранов П.Е.,Москва,2020-02-14,2020-02-22
6,Колесов С.П.,Москва,2020-02-01,2020-02-06
7,Лебедев Т.К.,Москва,2020-03-03,2020-03-06
8,Семенов И.В.,Москва,2020-01-23,2020-01-31
9,Колесов С.П.,Новосибирск,2020-06-03,2020-06-12


#Задание
Вывести название месяца и количество командировок для каждого месяца. Считаем, что командировка относится к некоторому месяцу, если она началась в этом месяце. Информацию вывести сначала в отсортированном по убыванию количества, а потом в алфавитном порядке по названию месяца виде. Название столбцов – Месяц и Количество.

In [11]:
sql_168 = """

SELECT

    date_format(date_first,'MMMM') AS Month
    , COUNT(*) AS Cnt

FROM trip

GROUP BY date_format(date_first,'MMMM') 

ORDER BY Cnt DESC, Month ASC
;

"""

spark.sql(sql_168).toPandas()

,Month,Cnt
0,February,4
1,January,4
2,June,3
3,May,3
4,April,2
5,July,2
6,March,2


#Задание
Вывести сумму суточных (произведение количества дней командировки и размера суточных) для командировок, первый день которых пришелся на февраль или март 2020 года. Значение суточных для каждой командировки занесено в столбец per_diem. Вывести фамилию и инициалы сотрудника, город, первый день командировки и сумму суточных. Последний столбец назвать Сумма. Информацию отсортировать сначала  в алфавитном порядке по фамилиям сотрудников, а затем по убыванию суммы суточных.

In [12]:
sql_169 = """

SELECT

    name
    , city
    , date_first
    , (DATEDIFF(date_last,date_first)*per_diem+per_diem) AS Summa

FROM trip

WHERE MONTH(date_first) IN (2,3)

ORDER BY name ASC, Summa DESC

"""

spark.sql(sql_169).toPandas()

,name,city,date_first,Summa
0,Абрамова К.А.,Москва,2020-02-23,5600
1,Баранов П.Е.,Москва,2020-02-14,6300
2,Колесов С.П.,Новосибирск,2020-02-27,6750
3,Колесов С.П.,Москва,2020-02-01,4200
4,Лебедев Т.К.,Москва,2020-03-03,2800
5,Семенов И.В.,Санкт-Петербург,2020-03-29,5600


#Задание
Вывести фамилию с инициалами и общую сумму суточных, полученных за все командировки для тех сотрудников, которые были в командировках больше чем 3 раза, в отсортированном по убыванию сумм суточных виде. Последний столбец назвать Сумма.

Только для этого задания изменена строка таблицы trip:

4	Ильиных Г.Р.	Владивосток	450	2020-01-12	2020-03-02

In [13]:
sql_170 = """

SELECT

    name
    , SUM( (DATEDIFF(date_last,date_first)*per_diem+per_diem)  ) AS Summa

FROM trip

WHERE name IN (SELECT name FROM trip GROUP BY name HAVING COUNT(*)>3 )

GROUP BY name

ORDER BY Summa DESC

"""

spark.sql(sql_170).toPandas()

,name,Summa
0,Абрамова К.А.,29200
1,Баранов П.Е.,21300
